In [143]:
import os
analysis_dir = ["PATH/TO/CSV/DATA.csv"]
for i in range(len(analysis_dir)):
    split_path = analysis_dir[i].split("/")
    analysis_dir[i] = os.path.join(*split_path)

In [144]:
def cleanup_and_create_file(path_file):
    with open(path_file, "r") as f:
        whole = f.readlines()
    header = whole[0][:-3].split(",,")
    unlabeled_full_headers = whole[1][:-2].split(",")
    assert 2*len(header) == len(unlabeled_full_headers)
    
    full_headers = []
    
    for i,cur_head in enumerate(unlabeled_full_headers):
        full_headers.append(cur_head+"_"+header[i//2])
    out_text = ",".join(full_headers)+"\n" 
    for i in range(2,len(whole)):
        if whole[i].strip() == "":
            break
        out_text+=whole[i][:-2]+"\n"
    split_by_dot = path_file.split(".")
    out_path = split_by_dot[0]+"_cleaned."+split_by_dot[1]
    with open(out_path, "w") as f:
        f.write(out_text)
    return out_path
    
    
out_dirs = [cleanup_and_create_file(cur_analysis) for cur_analysis in analysis_dir]

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import argrelextrema,find_peaks,find_peaks_cwt
from math import ceil
from lmfit import Model

DONOR = "Em570"
ACCEPTOR = "Em667"

fit_model = lambda t,a,b,tau: a*np.exp(-(t)/tau)+b

def generate_plot(csv_path_file, donor,acceptor,all_intensity=None):
    df = pd.read_csv(csv_path_file)
    lowest_cutoff = np.inf
    for i in range(0, len(df.columns), 2):
        time_col = df.columns[i]
        intensity_col = df.columns[i+1]
        #if donor in intensity_col:
        #    sns.lineplot(x=time_col, y=intensity_col, data=df)
        #last_peak = find_peaks(df[intensity_col], threshold=10)[0][-1]
        print(find_peaks(df[intensity_col], threshold=10)[0])
        peaks = find_peaks_cwt(df[intensity_col], 10,window_size=40,min_snr=1.5)
        if len(peaks) == 0:
            last_peak = np.inf
        else:
            last_peak = peaks[-1]
        
        
        #df = df.iloc[find_peaks(df[intensity_col], threshold=10)[0][-1]:,:]
        
        #df remove first 1% of data
        #df = df.iloc[ceil(df.shape[0]*0.01):,:] 
        
        #Line plot with dashed line
        
        cutoff_point = ceil(2*df.shape[0]/100) + last_peak
        
        
        sns.lineplot(x=time_col, y=intensity_col, data=df[df.index < cutoff_point],linestyle="--")
        
        sns.lineplot(x=time_col, y=intensity_col, data=df[df.index >= cutoff_point],label=intensity_col)
        
        #Put locations of peaks on plot
        #for cur_peak in peaks:
        #    plt.axvline(x=df[time_col].iloc[cur_peak],color="black",linestyle="--")
        
        if cutoff_point < lowest_cutoff:
            lowest_cutoff = cutoff_point
    plt.xlabel("Time (min)")
    plt.ylabel("Intensity (a.u.)")
    
    for cur_col in df.columns:
        if "Intensity" in cur_col:
            if donor in cur_col:
                donor_col = cur_col
            elif acceptor in cur_col:
                acceptor_col = cur_col
                
        elif "Time" in cur_col:
            if donor in cur_col:
                donor_time = cur_col
            elif acceptor in cur_col:
                acceptor_time = cur_col
    
    fret_efficiency = df[acceptor_col] / (df[donor_col]+df[acceptor_col])
    fret_time = (df[donor_time] + df[acceptor_time]) / 2
    plt.show()
    
    sns.lineplot(x=fret_time.iloc[lowest_cutoff:], y=fret_efficiency.iloc[lowest_cutoff:])
    
    valid = ~(np.isnan(fret_efficiency) | np.isnan(fret_time))
    
    
    
    
    #popt,_=curve_fit(fit_model, fret_time[valid], fret_efficiency[valid])
    #print(fret_efficiency[valid].iloc[lowest_cutoff:])
    emodel = Model(fit_model)
    params = emodel.make_params(a=1,b=0,tau=1)
    result = emodel.fit(fret_efficiency[valid].iloc[lowest_cutoff:], params, t=fret_time[valid].iloc[lowest_cutoff:])
    #Plot the fit model
    
    #plt.plot(np.arange(0,1000), fit_model(np.arange(0,1000), *popt), 'r-', label='fit')
    #print(popt)
    print(result.fit_report())
    plt.plot(fret_time[valid].iloc[lowest_cutoff:],result.best_fit,label="fit")
    plt.ylabel("FRET Efficiency")
    plt.xlabel("Time (min)")
    
    #Plot uncertainties as well
    perc_95 = result.eval_uncertainty(sigma=2)
    plt.fill_between(fret_time[valid].iloc[lowest_cutoff:], result.best_fit-perc_95, result.best_fit+perc_95, color="#ABABAB",
                 label='95% confidence interval')
    
    tau = result.best_values["tau"]
    #Plt text of tau, only to 2 decimal places
    plt.text(0.5,0.5,r"$\tau =$ "+str(round(tau,3)),transform=plt.gca().transAxes)
    
    
    plt.show()
    plt.cla()
    plt.clf()
    
    f_name = os.path.split(csv_path_file)[-1].split(".")[0]
    
    if all_intensity is not None:
        for i in range(0, len(df.columns), 2):
            time_col = df.columns[i]
            intensity_col = df.columns[i+1]
            sns.lineplot(x=time_col, y=intensity_col, data=df.iloc[df.index >= lowest_cutoff], label=intensity_col+"_"+f_name,ax=all_intensity)
        
    
    return result,lowest_cutoff,fret_efficiency,fret_time
        
def generate_global_plot(all_csvs, all_cutoffs):
    for j,csv in enumerate(all_csvs):
        df = pd.read_csv(csv)
        
        f_name = os.path.split(csv)[-1].split(".")[0]
        
        for i in range(0, len(df.columns), 2):
            time_col = df.columns[i]
            intensity_col = df.columns[i+1]
            sns.lineplot(x=time_col, y=intensity_col, data=df.iloc[df.index >= all_cutoffs[j]], label=intensity_col+"_"+f_name)
    plt.savefig("global_plot.png")
    plt.clf()
    
def tau_compare(all_csvs,taus):
    names = [os.path.split(csv)[-1].split(".")[0 ]for csv in all_csvs]
    
    df = pd.DataFrame({"Name":names,"Tau":taus})
    df["Unwinding"] = 1/ (df["Tau"] * 60)
    sns.barplot(x="Name",y="Tau",data=df)
    plt.savefig("tau_compare.png")
    plt.clf()
    sns.barplot(x="Name",y="Unwinding",data=df)
    plt.savefig("unwinding_compare.png")
    plt.clf()
    
def fret_compare(all_csvs,fes,fts,all_cutoffs):
    names = [os.path.split(csv)[-1].split(".")[0 ]for csv in all_csvs]
    for i,(ft,fe) in enumerate(zip(fts,fes)):
        sns.lineplot(x=ft.iloc[all_cutoffs[i]:],y=fe.iloc[all_cutoffs[i]:],label=names[i])
    plt.xlabel("Time (min)")
    plt.ylabel("FRET Efficiency")
    plt.savefig("fret_compare.png")
    plt.clf()
    
        
fig,all_intensity = plt.subplots(1,1)
all_cutoffs = []
taus = []
fes = []
fts = []
for out_dir in out_dirs:
    fit_result,lowest_cutoff,fret_efficiency,fret_time = generate_plot(out_dir, DONOR, ACCEPTOR,all_intensity)
    all_cutoffs.append(lowest_cutoff)
    taus.append(fit_result.best_values["tau"])
    fes.append(fret_efficiency)
    fts.append(fret_time)

generate_global_plot(out_dirs, all_cutoffs)
tau_compare(out_dirs,taus)
fret_compare(out_dirs,fes,fts,all_cutoffs)

ModuleNotFoundError: No module named 'pandas'